In [1]:
import os
import sys
import glob
import numpy as np
import pickle
from tqdm import tqdm

import pandas as pd

from sklearn.metrics import precision_score, roc_auc_score

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore")

In [2]:
import sys

sys.path.append('../pytorch')

In [3]:
from utils.fcd import *
from utils.crop import get_loader_crop

In [4]:
!nvidia-smi

Thu May 27 16:14:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64       Driver Version: 440.64       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:01:00.0 Off |                  N/A |
| 20%   39C    P8    16W / 250W |    765MiB / 11177MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
| 20%   26C    P8     7W / 250W |   1253MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [5]:
import imp
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F

import torch.optim as optim


import utils.data_processor as data_processor
imp.reload(data_processor)
from utils.data_processor import *

import utils.visualization_tools as visualization_tools
imp.reload(visualization_tools)
from utils.visualization_tools import *

import utils.metrics as metrics
imp.reload(metrics)
from utils.metrics import *

## Example of model evaluation for Crop dataset

In [6]:
DEVICE = 0

device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
torch.cuda.set_device(DEVICE)
torch.cuda.current_device()

0

In [22]:
from DGCNN import DGCNN_semseg
from train_eval import train_val, crop_val_step, iou

In [8]:
n_epochs = 1
num_point_cloud = 20000
batch_size = 1

In [9]:
result_dict = {"BCE_scratch": [], "IoU_scratch": [], "ROC-AUC_scratch": [], "Dice_scratch": []}

In [ ]:
for i in tqdm(range(26)):
    CFG = '../pytorch/cfgs/brain/brain_pospoolxyz.yaml'
    config = config_seting(CFG)
    train_data, test_data = get_loader_crop(config = config,
                                 num_points = num_point_cloud,
                                batch_size = 1,
                                 test_brain = i)
    path_to_model = f"models/crop/scratch/dgcnn_{i}.pth"
    
    model = DGCNN_semseg(k=20, emd_dims=512, dropout=0.5)
    model.load_state_dict(torch.load(path_to_model, map_location='cpu'))
    model.to(device)
    
    opt = optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-4)
    criterion = nn.BCEWithLogitsLoss()
    
    metrics_dict = train_val(model, opt, criterion, train_data, test_data, 
                             n_epochs=n_epochs, is_train=False, val_step_func=crop_val_step)
    
    result_dict["BCE_scratch"].append(metrics_dict["loss"])
    result_dict["IoU_scratch"].append(metrics_dict["iou"])
    result_dict["ROC-AUC_scratch"].append(metrics_dict["roc"])
    result_dict["Dice_scratch"].append(metrics_dict["dice"])
    
    del train_data, test_data
    del model

In [11]:
df_scratch = pd.DataFrame(result_dict, index=[f"{e}" for e in range(26)])

In [12]:
df_scratch.to_csv("metrics/crop/scratch.csv")